In [ ]:
import datetime
import json
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build

# Service Account Keyのロード
with open('credentials.json') as f:
    creds_data = json.load(f)
creds = Credentials.from_service_account_info(creds_data)

# load outlook items
with open('events.json','r') as f:
  cal = json.load(f)
items = cal['events']
calendar_id = cal['calender_id']
# print(calendar_id)
print(len(items),"loaded")

# Google Calendar APIのserviceを作成
service = build('calendar', 'v3', credentials=creds)

# get all google calendar items
event_result = (
    service.events()
    .list(
        calendarId=calendar_id,
        singleEvents=True,
        orderBy="startTime",
        maxResults=400
    )
    .execute()
)
events = event_result.get("items", [])

# time format functions
def timeformat(t):            
    ret = t.get("dateTime", t.get("date"))
    return datetime.datetime.strptime(ret[:-6], "%Y-%m-%dT%H:%M:%S")
def timeformatNozone(t):            
    ret = t.get("dateTime", t.get("date"))
    return datetime.datetime.strptime(ret, "%Y-%m-%dT%H:%M:%S")

def startEndToStr(start,end,zone):
  s = start.get("dateTime",start.get("date"))
  e = end.get("dateTime",end.get("date"),)
  if zone:
    st = timeformat(start)
    ed = timeformat(end)
  else:
    st = timeformatNozone(start)
    ed = timeformatNozone(end)
  return st.strftime('%Y/%m/%d %H:%M')+'-'+ed.strftime('%H:%M')

# Display fist 10 events in google calendar
# if not events:
#     print("No upcoming events found.")
# for i, event in enumerate(events):
#     start = timeformat(event["start"])
#     end = timeformat(event["end"]).time()
#     print("{:02}".format(i), startEndToStr(event["start"],event["end"],True), event["summary"])
#     if i > 10:
#         break
print("Found items on Gcal",len(events))

In [ ]:
from enum import IntEnum, auto

class Diff(IntEnum):
    SAME = auto()
    SUBJECT = auto()
    START = auto()
    END = auto()
    COLOR = auto()

def getEventIndex(events, item):
    for i, event in enumerate(events):
        if item["description"] == event["description"]:
            if item["summary"]!=event["summary"]:
                return i, Diff.SUBJECT
            if timeformatNozone(item["start"])!=timeformat(event["start"]):
                return i, Diff.START
            if timeformatNozone(item["end"])!=timeformat(event["end"]):
                return i, Diff.END
            if item.get("colorId")!=event.get("colorId"):
                return i, Diff.COLOR
            return i, Diff.SAME
    return -1, None
            
# check new/update 
for i, item in enumerate(items):
    index, df = getEventIndex(events,item)
    #print(i,(item["start"]),item["summary"])
    if index < 0:
        # new items
        print(i,"insert",startEndToStr(item["start"],item["end"],False),item["summary"])
        service.events().insert(
                calendarId=calendar_id,
                body=item).execute()
    else:
        event = events.pop(index)
        if df != Diff.SAME:
            # update event
            print(i,"update",startEndToStr(event["start"],event["end"],True),event["summary"])
            print(' ->',df, startEndToStr(item["start"],item["end"],False),item["summary"])
            service.events().update(
                    calendarId=calendar_id, 
                    eventId=event["id"], 
                    body=item
                ).execute()
# delete remaining event in list from google calendar
for event in events:
    print("delete",startEndToStr(event["start"],event["end"],True),event["summary"])
    service.events().delete(
        calendarId=calendar_id, 
        eventId=event["id"]).execute()

print("done")